[link text](https://www.kaggle.com/datasets/hazemfahmy/eye-gaze-detection/data)


In [ ]:
from google.colab import files
files.upload()  # This will prompt you to upload kaggle.json


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"esra25","key":"a415ec02e809410c89f784e2660ac6f7"}'}

In [ ]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

# Move the kaggle.json to the correct location
kaggle_json_path = '/mnt/data/kaggle.json'
os.environ['KAGGLE_CONFIG_DIR'] = '/mnt/data/'

# Use the Kaggle API to authenticate
api = KaggleApi()
api.authenticate()

# Example: List datasets available to you or get the dataset
# api.dataset_list()  # Uncomment to list available datasets


In [ ]:


# Set up Kaggle API credentials
os.environ['KAGGLE_CONFIG_DIR'] = '/mnt/data/'

# Initialize and authenticate the Kaggle API
api = KaggleApi()
api.authenticate()

# Define the dataset name
dataset_name = 'hazemfahmy/eye-gaze-detection'

# Download and unzip the dataset
api.dataset_download_files(dataset_name, path='/mnt/data/', unzip=True)



Dataset URL: https://www.kaggle.com/datasets/hazemfahmy/eye-gaze-detection


In [ ]:
# Path to the TrainingSet directory containing the eye images
dataset_directory = '/mnt/data/TrainingSet'  # Adjusted to the correct folder

# Load and preprocess the data from the TrainingSet directory
eye_images = load_and_preprocess_data(dataset_directory, target_size=(128, 128))

# Proceed with the rest of the steps (defining the autoencoder, training, etc.)




In [ ]:


# Step 1: Load and preprocess the eye images from the Kaggle dataset
def load_and_preprocess_data(directory, target_size=(128, 128)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Assuming images are in JPG or PNG format
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=target_size)  # Load image and resize
            img_array = img_to_array(img)  # Convert image to array
            img_array = img_array.astype("float32") / 255.0  # Normalize the image to [0, 1]
            images.append(img_array)

    return np.array(images)

# Load eye images from the Kaggle dataset (change this path to your actual directory)
dataset_directory = '/mnt/data/eye_gaze_data/eye-gaze-detection/TrainingSet'  # Replace this with your actual dataset path
eye_images = load_and_preprocess_data(dataset_directory, target_size=(128, 128))

# Step 2: Define the autoencoder architecture
input_shape = (128, 128, 3)  # Input image size for color images (RGB)
input_img = Input(shape=input_shape)

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Flatten()(x)
latent_vector = Dense(128, activation='relu')(x)  # Latent space with 128 units

# Decoder
x = Dense(128 * 128 * 64, activation='relu')(latent_vector)
x = Reshape((128, 128, 64))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
output_img = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

# Define the autoencoder model
autoencoder = Model(inputs=input_img, outputs=output_img)

# Compile the model with Mean Squared Error loss
autoencoder.compile(optimizer='adam', loss='mse')

# Display the model summary
autoencoder.summary()

# Step 3: Train the autoencoder (use eye images for both input and output)
history = autoencoder.fit(
    eye_images, eye_images,  # Use the same images for both input and output (autoencoder)
    epochs=10,
    batch_size=32,
    shuffle=True
)

# Step 4: Visualize the results
def display_reconstructed_images(model, test_images, num_images=10):
    decoded_imgs = model.predict(test_images[:num_images])
    plt.figure(figsize=(20, 4))
    for i in range(num_images):
        # Original images
        ax = plt.subplot(2, num_images, i + 1)
        plt.imshow(test_images[i])
        plt.axis("off")

        # Reconstructed images
        ax = plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(decoded_imgs[i])
        plt.axis("off")

    plt.show()

# Display a few original and reconstructed images
display_reconstructed_images(autoencoder, eye_images)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/eye_gaze_data/eye-gaze-detection/TrainingSet'

In [ ]:
# Step 1: Create an ImageDataGenerator instance and resize images
train_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0, 1]

# Ensure images are resized to (128, 128) when loaded
train_generator = train_datagen.flow_from_directory(
    'eye_gaze_data/TrainingSet/',  # Path to your data folder
    target_size=(128, 128),        # Resize all images to (128, 128)
    batch_size=32,                 # Batch size for training
    class_mode=None,               # No labels for autoencoder
    shuffle=True                   # Shuffle data if needed
)

# Step 2: Check the data being returned from the generator before training
for data in train_generator:
    if data is None:
        print("Found None in generator!")
        break  # Stop if we find None
    print("Full batch shape:", data[0].shape)  # Full shape of the data batch
    print("First image batch shape:", data[0][0].shape)  # Shape of the first image
    print("Data type:", type(data))  # Ensure it's a valid array or tensor
    break  # Stop after inspecting the first batch

# Step 3: Define the autoencoder architecture
input_shape = (128, 128, 3)  # Input image size
input_img = Input(shape=input_shape)

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Flatten()(x)
latent_vector = Dense(128, activation='relu')(x)

# Decoder
x = Dense(128 * 128 * 64, activation='relu')(latent_vector)
x = Reshape((128, 128, 64))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
output_img = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

# Define the autoencoder model
autoencoder = Model(inputs=input_img, outputs=output_img)

# Compile the model with Mean Squared Error loss
autoencoder.compile(optimizer='adam', loss='mse')

# Display the model summary
autoencoder.summary()

# Step 4: Proceed with training only if the generator is valid
history = autoencoder.fit(
    train_generator,  # Directly pass the generator
    steps_per_epoch=len(train_generator),
    epochs=10  # Adjust based on your needs
)


Found 61073 images belonging to 1 classes.
Full batch shape: (128, 128, 3)
First image batch shape: (128, 3)
Data type: <class 'numpy.ndarray'>


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 128, 128, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 1048576)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │     134,217,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1048576)             │     135,266,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_4 (Reshape)                  │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_12                  │ (None, 128, 128, 64)        │          36,928 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_13                  │ (None, 128, 128, 32)        │          18,464 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_14                  │ (None, 128, 128, 3)         │             867 │
│ (Conv2DTranspose)                    │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 269,559,811 (1.00 GB)

 Trainable params: 269,559,811 (1.00 GB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


ValueError: None values not supported.

In [ ]:
# Ensure the model is compiled before training
autoencoder.compile(optimizer='adam', loss='mse')

# Check the data being returned from the generator before training
for data in train_generator:
    if data is None:
        print("Found None in generator!")
        break  # Stop if we find None
    print("Full batch shape:", data[0].shape)  # Full shape of the data batch
    print("First image batch shape:", data[0][0].shape)  # Shape of the first image
    print("Data type:", type(data))  # Ensure it's a valid array or tensor
    break  # Stop after inspecting the first batch

# Proceed with training only if the generator is valid
history = autoencoder.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10  # Adjust based on your needs
)


Full batch shape: (32, 128, 128, 3)
First image batch shape: (128, 128, 3)
Data type: <class 'tuple'>
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/mse/sub defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.

In [ ]:
# Step 1: Create an ImageDataGenerator instance and resize images
train_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0, 1]

# Ensure images are resized to (128, 128) when loaded
train_generator = train_datagen.flow_from_directory(
    'eye_gaze_data/TrainingSet/',  # Path to your data folder
    target_size=(128, 128),        # Resize all images to (128, 128)
    batch_size=32,                 # Adjust batch size for training
    class_mode='binary'            # Adjust this based on your label format (e.g., 'binary' or 'categorical')
)

# Step 2: Check the data being returned from the generator before training (This is your existing check)
for data in train_generator:
    if data is None:
        print("Found None in generator!")
        break  # Stop if we find None
    print("Full batch shape:", data[0].shape)  # Check full shape of the data batch
    print("First image batch shape:", data[0][0].shape)  # Check the shape of the first image
    print("Data type:", type(data))  # Ensure it's a valid array or tensor
    break  # Stop after inspecting the first batch

# Step 3: Ensure train_generator is valid before proceeding to training
if train_generator is None:
    print("Error: train_generator is None!")
else:
    # Proceed with training only if the generator is valid
    history = autoencoder.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=10  # Adjust based on your needs
    )


Found 61073 images belonging to 1 classes.
Full batch shape: (32, 128, 128, 3)
First image batch shape: (128, 128, 3)
Data type: <class 'tuple'>
Epoch 1/10


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [ ]:
# Ensure train_generator is not None
for data in train_generator:
    if data is None:
        print("Error: Found None data in generator")
        continue  # This will skip the current batch and move to the next one
    print("Batch shape:", data[0].shape)  # Print the shape of the image batch
    break  # Stop after inspecting the first batch


Batch shape: (128, 128, 3)


Create and Train an Autoencoder

---



In [ ]:


# Define the input shape
input_shape = (128, 128, 3)  # Adjust based on your image size
input_img = Input(shape=input_shape)

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Flatten()(x)
latent_vector = Dense(128, activation='relu')(x)

# Decoder
x = Dense(128 * 128 * 64, activation='relu')(latent_vector)
x = Reshape((128, 128, 64))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
output_img = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

# Define the autoencoder model
autoencoder = Model(inputs=input_img, outputs=output_img)
autoencoder.compile(optimizer='adam', loss='mse')

# Display the model summary
autoencoder.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 128, 128, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1048576)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │     134,217,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1048576)             │     135,266,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_1 (Reshape)                  │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_3 (Conv2DTranspose) │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_4 (Conv2DTranspose) │ (None, 128, 128, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_5 (Conv2DTranspose) │ (None, 128, 128, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 269,559,811 (1.00 GB)

 Trainable params: 269,559,811 (1.00 GB)

 Non-trainable params: 0 (0.00 B)

Train the Autoencoder


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(rescale=1.0 / 255.0)  # Normalize pixel values to [0, 1]

# Define the path to the TrainingSet directory
training_set_path = './eye_gaze_data/TrainingSet/TrainingSet'

# Create the generator
train_generator = datagen.flow_from_directory(
    training_set_path,
    target_size=(128, 128),  # Resize images to 128x128
    batch_size=32,          # Number of images per batch
    class_mode=None         # Autoencoder doesn't need class labels
)

print(f"Train generator created. Total samples: {train_generator.n}")


Found 61073 images belonging to 8 classes.
Train generator created. Total samples: 61073


In [ ]:
print(f"Number of samples in generator: {train_generator.n}")
print(f"Batch size: {train_generator.batch_size}")
print(f"Steps per epoch: {len(train_generator)}")


Number of samples in generator: 61073
Batch size: 32
Steps per epoch: 1909


In [ ]:
# Step 4: Train the autoencoder
# Check the data being returned from the generator before training
for data in train_generator:
    if data is None:
        print("Found None in generator!")
        break  # Stop if we find None
    print("Batch shape:", data[0].shape)  # Check shape of data (images)
    print("First image batch:", data[0][0])  # Inspect the first image in the batch
    break  # Inspect only the first batch

history = autoencoder.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10  # Adjust based on your needs
)




Batch shape: (128, 128, 3)
First image batch: [[0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.38431376 0.38431376]
 [0.38431376 0.384

ValueError: None values not supported.